# My Capstone Project
## By Sohail Ladha
### The goal is to use this notebook to complete the Capstone Project and share on Github

In [10]:
import pandas as pd
import numpy as np

In [11]:
print("Hello Capstone Project Course!")

Hello Capstone Project Course!


## Week 3 Assignment
## Segmenting and Clustering Neighborhoods in the city of Toronto, Canada

In [12]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json 

from geopy.geocoders import Nominatim 

import requests 
from pandas.io.json import json_normalize

import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes 
import folium 

print('Libraries imported.')

Libraries imported.


In [13]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
df_toronto = pd.read_html(url, attrs={"class": "wikitable"})[0]
df_toronto.head()

,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [14]:
# ignore cells with a borough that is not assigned.

df_toronto = df_toronto[df_toronto.Borough != 'Not assigned'].reset_index(drop=True)
df_toronto.head()

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


##### Making sure the Postal Codes are unique and there are no 'Not assigned' Neighborhoods.

In [25]:
len(df_toronto['Postal Code'].unique())

103

In [26]:
# making sure there are no 'Not assigned' values in df_toronto

checkx = 'Not assigned' in df_toronto
print(checkx)

False


In [27]:
df_toronto.shape

(103, 3)